<a href="https://colab.research.google.com/github/Svetorus/Spark/blob/main/Spark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark

In [ ]:
df=spark.read.option('interSchema','true').option('header','true').csv('owid-covid-data.csv')
# df = spark.read.csv('owid-covid-data.csv',header=True)

from pyspark.sql.functions import col,lit
import pyspark.sql.functions as func
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# *Pandas*

In [ ]:
# библиотеки
import pandas as pd
import numpy as np
from google.colab import files

In [ ]:
# загрузка из компа в google colab
uploaded = files.upload()

Saving owid-covid-data.csv to owid-covid-data.csv


In [ ]:
covid = pd.read_csv('owid-covid-data.csv')
covid.head(3)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511


In [ ]:
covid.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
       'new_vaccinations_smoothed', 'total_vaccinations_per_hun

# 1. Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)  
### *Pandas*

In [ ]:
covid[['iso_code','location','reproduction_rate']].loc[covid['date']=='2020-03-31',]\
  .nlargest(15, 'reproduction_rate')

,iso_code,location,reproduction_rate
60168,QAT,Qatar,2.49
7234,BLR,Belarus,2.48
61028,RUS,Russia,2.46
75850,TUR,Turkey,2.40
33862,IND,India,2.18
58927,PHL,Philippines,2.15
76733,UKR,Ukraine,2.08
77178,ARE,United Arab Emirates,2.07
10574,BRA,Brazil,2.02
37775,JPN,Japan,2.02


### PySpark

In [ ]:
df.select('iso_code','location','reproduction_rate').filter( (col('date') == lit('2020-03-31')) )\
  .orderBy(col("reproduction_rate").desc()).show(15)

# 2. Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)  
### *Pandas*

In [ ]:
covid.loc[(covid['date']<='2021-03-31') & (covid['date']>='2021-03-29')].groupby(['location'],as_index=False)\
  .aggregate({'new_cases':'max'}).nlargest(10, 'new_cases')

,location,new_cases
210,World,683205.0
63,Europe,255985.0
64,European Union,216452.0
10,Asia,183350.0
179,South America,145976.0
27,Brazil,90638.0
143,North America,84748.0
89,India,72330.0
203,United States,69429.0
69,France,59054.0


In [ ]:
df= covid.loc[(covid['date']<='2021-03-31') & (covid['date']>='2021-03-29')]\
    .groupby(['location'],as_index=False).aggregate({'new_cases':'max'})
covid_2 = covid[['date','location','new_cases']].loc[(covid['date']<='2021-03-31') & (covid['date']>='2021-03-29')]
new_covid = covid_2.merge( df,  how='inner', on=['location','new_cases'])
new_covid.nlargest(10, 'new_cases')

,date,location,new_cases
274,2021-03-31,World,683205.0
77,2021-03-31,Europe,255985.0
78,2021-03-31,European Union,216452.0
12,2021-03-31,Asia,183350.0
235,2021-03-31,South America,145976.0
33,2021-03-31,Brazil,90638.0
193,2021-03-31,North America,84748.0
119,2021-03-31,India,72330.0
263,2021-03-29,United States,69429.0
89,2021-03-31,France,59054.0


In [ ]:
#covid.loc[(covid['date']<='2021-03-31') & (covid['date']>='2021-03-29')]\
#  .groupby(['date', 'location'])['new_cases'].agg(['sum'])

### PySpark

In [ ]:
df_1 =df.filter( (col('date') >= lit('2021-03-29')) & (col('date') <= lit('2021-03-31')) )\
  .groupby('location').agg({"new_cases": "max"})
df_1 = df_1.withColumnRenamed('max(new_cases)', 'new_cases')
covid_2 = df.select('date','location','new_cases')\
  .filter( (col('date') >= lit('2021-03-29')) & (col('date') <= lit('2021-03-31')) )
new_covid = covid_2.join( df_1,  on=['location','new_cases'] , how='inner')
new_covid.select('date','location','new_cases').orderBy(col('new_cases').desc()).show(10)

# 3. Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)  
### *Pandas*

In [ ]:
covid['yesterday'] = covid.loc[(covid['date']<='2020-03-31') & (covid['date']>='2020-03-24'),['new_cases']]\
  .shift(periods=1, fill_value=0)
covid['diffirent'] = covid['new_cases']-covid['yesterday']
covid.loc[(covid['date']<='2020-03-31') & (covid['date']>='2020-03-24' ) & (covid['location'] == 'Russia'),\
          ['date','location','new_cases','yesterday','diffirent']]

,date,location,new_cases,yesterday,diffirent
61021,2020-03-24,Russia,57.0,136.0,-79.0
61022,2020-03-25,Russia,163.0,57.0,106.0
61023,2020-03-26,Russia,182.0,163.0,19.0
61024,2020-03-27,Russia,196.0,182.0,14.0
61025,2020-03-28,Russia,228.0,196.0,32.0
61026,2020-03-29,Russia,270.0,228.0,42.0
61027,2020-03-30,Russia,302.0,270.0,32.0
61028,2020-03-31,Russia,501.0,302.0,199.0


### PySpark

In [ ]:
df_3 = df.filter( (col('date') >= lit('2021-03-29')) & (col('date') <= lit('2021-03-31')) )\
  .withColumn('yesterday', F.lag(col('new_cases'),int=1).over(Window.partitionBy('new_cases')
df_3 = df.withColumn('diffirent', df.new_cases - df.yesterday)
df_3 = df.select('date','location','new_cases','yesterday','diffirent')\
  .filter( (col('date') >= lit('2021-03-29')) & (col('date') <= lit('2021-03-31'))  & (col('location') == 'Russia') ).show()